In [1]:
from env_2 import UArmEnv
from ultralytics import YOLO
from stable_baselines3 import PPO
import time

# Initialise environment
env = UArmEnv(render=True)
env.reset()

# Step 1: Move to top-down camera view
topdown_action = [0.0, 0.8, -1.57, 0]
for _ in range(10):
    env.step(topdown_action)

# Step 2: Capture image
env.get_camera_image()
env.capture_image_label_and_gt("test_dataset", image_index=0)
image_path = "test_dataset/images/image_0000.png"

# Step 3: Run YOLO prediction
yolo_model = YOLO("runs/detect/train3/weights/best.pt")  # use your best model
results = yolo_model(image_path)

# Coordinate bounds for reconversion
X_MIN, X_MAX = -0.170, 0.210
Y_MIN, Y_MAX = -0.490, -0.120

# Convert YOLO predictions to world coordinates
predicted_boxes = []
for box in results[0].boxes:
    cls = int(box.cls[0])
    x_center_n, y_center_n = map(float, box.xywhn[0][:2])

    x_world = X_MIN + (1.0 - x_center_n) * (X_MAX - X_MIN)
    y_world = Y_MIN + y_center_n * (Y_MAX - Y_MIN)

    predicted_boxes.append((cls, x_world, y_world))

# Define goal positions by class
goal_positions = {
    0: env.goal_pos_red,
    1: env.goal_pos_green,
    2: env.goal_pos_blue
}

# Step 4: Iterate through all predictions

ppo_model = PPO.load("ppo_uarm_final_20")

def move_to(x, y, z):
    env.set_goal([x, y, z])  # hover height
    obs = env._get_obs()

    done = False
    while not done:
        action, _ = ppo_model.predict(obs)
        obs, reward, done, info = env.step(action)
        # print("Action taken:", action)
        time.sleep(0.05) 

for cls, x, y in predicted_boxes:
    print(f"🔄 Handling Class {cls} at ({x:.5f}, {y:.5f})")

    # Step 4.1: Move above box
    move_to(x, y, 0.025)  # hover height
    env.toggle_gripper_func()
    # time.sleep(0.2) 

    move_to(x, y, 0.06)  # lift height

    move_to(*goal_positions[cls])
    env.toggle_gripper_func()
    # time.sleep(0.2) 
    

print("✅ All boxes sorted!")
env.close()


pybullet build time: Jan 29 2025 23:16:28


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3060 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 550.163.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 550.163.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3060 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
✅ Saved image, label, and ground truth for image_0000

image 1/1 /home/fatduck/git/ai_project/main/UArm_robot_training/

/home/fatduck/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Gripper closed.
Goal manually set to: [    0.10993    -0.24999        0.06]
📦 Box snapped to gripper!
Goal manually set to: [   -0.16939     -0.2906        0.03]
Gripper opened.
🔄 Handling Class 2 at (0.03991, -0.24942)
Goal manually set to: [   0.039907    -0.24942       0.025]
👐 Box released.
b3Printf: b3Warning[examples/SharedMemory/PhysicsClientSharedMemory.cpp,808]:

b3Printf: removeConstraint failed
Gripper closed.
Goal manually set to: [   0.039907    -0.24942        0.06]
📦 Box snapped to gripper!
Goal manually set to: [   -0.16939     -0.2906        0.03]
Gripper opened.
🔄 Handling Class 0 at (0.03936, -0.32018)
Goal manually set to: [   0.039359    -0.32018       0.025]
👐 Box released.
b3Printf: b3Warning[examples/SharedMemory/PhysicsClientSharedMemory.cpp,808]:

b3Printf: removeConstraint failed
Gripper closed.
Goal manually set to: [   0.039359    -0.32018        0.06]
📦 Box snapped to gripper!
Goal manually set to: [   -0.16939     -0.1506        0.03]
Gripper opened.
🔄 Ha